# Задание 1 (7 балла)
Выберите одного певца. Скачайте несколько его свежих портретных фото. Какой ему возраст в среднем дает апи?

Делайте запросы в цикле.

Если не знаете, где взять легально mp3 файлы, то можно воспользоваться этим сервисом: https://www.jamendo.com/start

### Microsoft Face API:
https://www.microsoft.com/cognitive-services/en-us/face-api - дают 7-дневный пробный период без кредитки
    
https://docs.microsoft.com/ru-ru/azure/cognitive-services/face/quickstarts/python - quick start

In [5]:
# Выберите одного певца. Скачайте несколько его свежих портретных фото. Какой ему возраст в среднем дает апи?

import requests
from io import BytesIO
import numpy

subscription_key = '10abe93f74344a4b81eddcedb5481f37'
assert subscription_key

face_api_url = 'https://westcentralus.api.cognitive.microsoft.com/face/v1.0/detect'

headers = {'Ocp-Apim-Subscription-Key': subscription_key, 
           'Content-Type': 'application/octet-stream'}
params = {
    'returnFaceId': 'true',
    'returnFaceLandmarks': 'false',
    'returnFaceAttributes': 'age'
}

img_path = 'thom_yorke/thom_yorke'

ages = []
for i in range(1,7):
    photo_path = "{}{}.jpg".format(img_path,i)
    with open(photo_path, "rb") as image_file:
        data = image_file.read()
        
    response = requests.post(face_api_url, params=params, headers=headers, data=data)
    faces = response.json()
    for face in faces:
        fa = face["faceAttributes"]
        ages.append(int(fa["age"]))
           
print('Средний возраст человека на фото: {}'.format(numpy.mean(ages)))

Средний возраст человека на фото: 51.5


# Задание 2 (7 балла)
Выберите одну его песню. Разбейте ее на кусочки. Кусочки не должны быть больше 1 мб и длиннее 1 минуты. Получите текст кусочков с помощью соответствующего апи.

Для английского языка используйте en-US, для русского ru-RU. Яндекс русские песни понимает лучше.

# Yandex SpeechKit
https://tech.yandex.ru/speechkit/

Нас не интересует встраивание этой технологии в мобильное приложение. Так что идем в https://tech.yandex.ru/speechkit/cloud/ . Отсюда в кабинет разработчика: https://developer.tech.yandex.ru и получаем там API ключ на 30 дней.

С новеньким ключем читаем документацию: https://tech.yandex.ru/speechkit/cloud/doc/guide/concepts/speechkit-cloud-asr-quick-start-docpage/

Проблематично что-то понять, но гугление дает прекрасный пример на Питоне: https://toster.ru/q/310132

In [4]:
import requests
from bs4 import BeautifulSoup
from pydub import AudioSegment
import xml.etree.ElementTree as ET

key = 'ed49bebf-4b94-41bd-9314-5166fca9c80e'
uuid = '0b6831a499783774116100aacacd8545' # пришлось прочесть документацию чтоб понять что это важно =)
url = 'https://asr.yandex.net/asr_xml?key=' + key + '&uuid=' + uuid + '&topic=queries&lang=ru-RU'
headers = {"Content-type": 'audio/x-mpeg-3'}


# сегментация не работает =[, пришлось искать короткий файл
# да и судя по документации эта штука предназначена для кородких команд
# но, понимаю, если есть штука почему бы ее не использовать для всего ^___^ 

#sound = AudioSegment.from_mp3("thom_yorke/radiohead-climbing-up-the-walls.mp3")
#halfway_point = len(sound) / 5
#second_half = sound[halfway_point:]

#print(halfway_point)

data = open('pilot/pilot-stikh.mp3', 'rb')
    
response = requests.post(url, headers=headers, data=data)

result = "Произошла ошибка"
if response.status_code == 200:
    root = ET.fromstring(response.text)
    result = root[0].text

print(result)

чистый свет обычный начинался день народ ломился за портвейном жевали дети колбасу в пропавшей коньяком кофейной гаишник ковырял в носу белье сушилка на балконе настройки догорал сарай иван никифорович крюгер попал под рельсы и трамвай земли по речке девушка плыла кому то рядом билли рыло а заодно в колокола


In [61]:
with open('train_text/ttest.txt', 'w') as f:
        f.write(result)

# Задание 3 по желанию.

Получившиеся тексты классифицируйте с помощью fasttext.

Для русского языка вам потребуется найти самостоятельно корпус текстов. Они есть в интернете, например, вот по твиттеру: http://study.mokoron.com/


### FastText
https://fasttext.cc

In [59]:
import pandas as pd

column_names = ['id','tdate','tname','ttext','ttype','trep','trtw','tfav', 'tstcount', 'tfol', 'tfrien', 'listcount']
df_posetive = pd.read_csv('train_text/positive.csv', sep= ';', header=None)
df_posetive.columns = column_names
df_negative = pd.read_csv('train_text/negative.csv', sep= ';', header=None)
df_negative.columns = column_names

df_train = df_posetive.append(df_negative, ignore_index=True)
train_labels = df_train.ttype
train_data = df_train.ttext

with open('train_text/ttrain.txt', 'w') as f:
    for i in range(df_train.shape[0]):
        f.write(('__label__{} {}\n'.format(train_labels[i], train_data[i])))



In [67]:
# /ПУТЬ ДО РЕПОЗИТОРИЯ/home_works/
!/Users/andreysinetskiy/GitHub/python/home_works/fasttext supervised -input train_text/ttrain.txt -output model_sent

Read 0M words
Number of words:  915
Number of labels: 1
Progress: 100.0% words/sec/thread:    6504 lr:  0.000000 loss:  0.000000 ETA:   0h 0m


In [69]:
!/Users/andreysinetskiy/GitHub/python/home_works/fasttext predict model_sent.bin train_text/ttest.txt

__label__1
